# Social characteristics of the Marvel Universe

In [ ]:
sc

In [ ]:
spark

## Read in the data.

Run the right cell depending on the platform being used.

For AWS EMR use:

In [ ]:
file = sc.textFile("s3://bigdatateaching/marvel/porgat.txt")

For Databricks on Azure use:

In [ ]:
file = sc.textFile("wasbs://marvel@bigdatateaching.blob.core.windows.net/porgat.txt")

## Clean The Data

The data file is in three parts, with the single file:

* Marvel Characters
* Publications
* Relationships between the two

We need to pre-process the file before we can use it. The following operations are all RDD operations.

Let's look at the file.

Count the number of records in the file.

Define a new RDD that removes the headers from the file. The headers are lines that begin with a star.

In [ ]:
noHeaders = file.

Look at the first 5 records of noHeaders

In [ ]:
noHeaders.

Extract a pair from each line:  the leading integer and a string for the rest of the line

In [ ]:
paired = noHeaders.

Filter relationships as they do not start with quotes, then split the integer list

In [ ]:
scatteredRelationships = paired.

Relationships for the same character id sometime spans more than a line in the file, so let's group them together

In [ ]:
relationships = scatteredRelationships.

Filter non-relationships as they start with quotes ; remove the quotes

In [ ]:
nonRelationships = paired.

Characters stop at a certain line (part of the initial header ; we hardcode it here)

In [ ]:
characters = nonRelationships.

Publications starts after the characters

In [ ]:
publications = nonRelationships.

The following cells begin to use SparkSQL. 

Spark SQL works with Data Frames which are a kind of “structured” RDD or an “RDD with schema”.

The integration between the two works by creating a RDD of Row (a type from pyspark.sql) and then creating a Data Frame from it.

The Data Frames can then be registered as views.  It is those views we’ll query using Spark SQL.

In [ ]:
from pyspark.sql import Row

Let's create dataframes out of the RDDs and register them as temporary views for SQL to use


In [ ]:
#  Relationships has a list as a component, let's flat that
flatRelationships = relationships.flatMap(lambda (charId, pubList):  [(charId, pubId) for pubId in pubList])

In [ ]:
#  Let's map the relationships to an RDD of rows in order to create a data frame out of it
relationshipsDf = spark.createDataFrame(flatRelationships.map(lambda t: Row(charId=t[0], pubId=t[1])))


In [ ]:
#  Register relationships as a temporary view
relationshipsDf.createOrReplaceTempView("relationships")


In [ ]:
#  Let's do the same for characters
charactersDf = spark.
charactersDf.

In [ ]:
#  and for publications
publicationsDf = spark.
publicationsDf.

In [ ]:
relationshipsDf.show(10)

The following cell is the standard way of running a SQL query on Spark. This query ranks Marvel characters in duo in order of join-appearances in publications. 

In [ ]:
df1 = spark.sql("""SELECT c1.name AS name1, c2.name AS name2, sub.charId1, sub.charId2, sub.pubCount
FROM
(
  SELECT r1.charId AS charId1, r2.charId AS charId2, COUNT(r1.pubId, r2.pubId) AS pubCount
  FROM relationships AS r1
  CROSS JOIN relationships AS r2
  WHERE r1.charId < r2.charId
  AND r1.pubId=r2.pubId
  GROUP BY r1.charId, r2.charId
) AS sub
INNER JOIN characters c1 ON c1.charId=sub.charId1
INNER JOIN characters c2 ON c2.charId=sub.charId2
ORDER BY sub.pubCount DESC
LIMIT 10""").cache()


In [ ]:
df1.take(10)

In [ ]:
df2 = spark.sql("""
SELECT c1.name AS name1, c2.name AS name2, c3.name AS name3, sub.charId1, sub.charId2, sub.charId3, sub.pubCount
FROM
(
  SELECT r1.charId AS charId1, r2.charId AS charId2, r3.charId AS charId3, COUNT(r1.pubId, r2.pubId, r3.pubId) AS pubCount
  FROM relationships AS r1
  CROSS JOIN relationships AS r2
  CROSS JOIN relationships AS r3
  WHERE r1.charId < r2.charId
  AND r2.charId < r3.charId
  AND r1.pubId=r2.pubId
  AND r2.pubId=r3.pubId
  GROUP BY r1.charId, r2.charId, r3.charId
) AS sub
INNER JOIN characters c1 ON c1.charId=sub.charId1
INNER JOIN characters c2 ON c2.charId=sub.charId2
INNER JOIN characters c3 ON c3.charId=sub.charId3
ORDER BY sub.pubCount DESC
LIMIT 10
""").cache()


In [ ]:
df2.show(10)

In [ ]:
sc.stop()

This lab was adapted from [https://vincentlauzon.com/2018/01/24/azure-databricks-spark-sql-data-frames/](https://vincentlauzon.com/2018/01/24/azure-databricks-spark-sql-data-frames/)

Saving a DataFrame to a csv
```
publicationsDf.write\
    .format("com.databricks.spark.csv")\
    .option("header", "true")\
    .save("s3://bigdatateaching/marvel/publication")
```